## NER - Natasha

Natasha  - питоновская библиотека для извлечения именованных сущностей. Она похоже на Tomita-parser, но в ней все на чистом питоне, с открытым кодом и активно развивается. 

Если быть точнее, то natasha - набор готовых правил для парсера yargy. 

Есть например готовые правила для извлечения персон.

In [2]:
# Установить можно через pip.
from natasha import NamesExtractor

In [3]:
sents = open('sents.txt', encoding='utf-8').read().splitlines()

In [4]:
extractor = NamesExtractor()

Видно, что извлекается не очень хорошо. К тому же в реальной задаче нам потребуется извлекать ещё какие-то аттрибуты сущностей и группировать их в факты. И не обязательно этих типов. Поэтому полезно разобраться с самим парсером yargy.

Документация yargy: https://yargy.readthedocs.io/ru/latest/reference.html

Лучше всего с такими штуками разбираться на практике. Давайте попробуем написать правила для извлечения персон. Каждая персона должна описываться 3 полями - Имя, Фамилия, Отчество. Также у Персоны должны быть атрибуты - должность и место работы. 

За основу возьмем пример из документации:

In [11]:
from yargy import Parser, rule, or_
from yargy.predicates import gram
from yargy.pipelines import morph_pipeline
from yargy.interpretation import fact
from IPython.display import display

Person = fact(
    'Person',
    ['position', 'name']
)
Name = fact(
    'Name',
    ['first', 'last']
)


POSITION = morph_pipeline([
    'премьер министр',
    'президент'
])

NAME = rule(
    gram('Name').interpretation(
        Name.first.inflected()
    ),
    gram('Surn').interpretation(
        Name.last.inflected()
    )
).interpretation(
    Name
)

PERSON = rule(
    POSITION.interpretation(
        Person.position.inflected()
    ),
    NAME.interpretation(
        Person.name
    )
).interpretation(
    Person
)


parser = Parser(PERSON)

Посмотрим, что получается:

In [32]:
# for sent in sents[:100]:
#     print(sent)
#     for match in parser.findall(sent):
#         display(match.fact)
#     print('---------------')

Чтобы проверить какие морфологические тэги приписываются словам, можно использовать MorphTokenizer.

In [33]:
from yargy.tokenizer import MorphTokenizer
tokenizer = MorphTokenizer()

In [9]:
list(tokenizer('Медведев'))

[MorphToken('Медведев',
            [0, 8),
            'RU',
            [Form('медведев', Grams(NOUN,Sgtm,Surn,anim,masc,nomn,sing))])]

## Задача на семинар:
Доработать правила так, чтобы извлекалось как можно больше правильных фактов.

In [30]:
Person = fact(
    'Person',
    ['position', 'name', 'place']
)
Name = fact(
    'Name',
    ['first', 'last', 'patronymic']
)


POSITION = morph_pipeline([
    'премьер министр',
    'президент',
    'премьер-министр',
    'министр',
    'вице-премьер',
    'премьер',
    'министр финансов',
    'политик',
    'министр внутренних дел',
    'министр обороны',
    'министр культуры',
    'пресс-сектретарь',
    'канцлер',
    'председатель'
])

NAME = or_(
    rule(
    gram('Name').interpretation(
        Name.first.inflected()
    ),
    gram('Surn').interpretation(
        Name.last.inflected()
    ),
    gram('Patr').interpretation(
        Name.patronymic.inflected()
    ).optional()
).interpretation(
        Name
),
    rule(
    gram('Name').repeatable().interpretation(
        Name.last.inflected())
).interpretation(
        Name
))

PLACE = rule(
    gram('Geox').interpretation(
            Person.place.normalized()
).optional()
)

PERSON = rule(
    POSITION.interpretation(
        Person.position.normalized()
    ),
    PLACE,
    NAME.interpretation(
        Person.name
    )
).interpretation(
    Person
)




parser = Parser(PERSON)

In [31]:
matches = []

for sent in sents:
    for match in parser.findall(sent):
        matches.append(match.fact)


len(matches)

71

In [35]:
list(tokenizer('премьер-министр России Дмитрий Медведев'))

[MorphToken('премьер',
            [0, 7),
            'RU',
            [Form('премьер', Grams(NOUN,anim,masc,nomn,sing)),
             Form('премьера', Grams(NOUN,femn,gent,inan,plur))]),
 Token('-', [7, 8), 'PUNCT'),
 MorphToken('министр',
            [8, 15),
            'RU',
            [Form('министр', Grams(NOUN,anim,masc,nomn,sing))]),
 MorphToken('России',
            [16, 22),
            'RU',
            [Form('россия', Grams(Geox,NOUN,Sgtm,femn,gent,inan,sing)),
             Form('россия', Grams(Geox,NOUN,Sgtm,femn,inan,loct,sing)),
             Form('россия', Grams(Geox,NOUN,Sgtm,datv,femn,inan,sing))]),
 MorphToken('Дмитрий',
            [23, 30),
            'RU',
            [Form('дмитрий', Grams(NOUN,Name,anim,masc,nomn,sing)),
             Form('дмитрия', Grams(NOUN,Name,anim,femn,gent,plur)),
             Form('дмитрия', Grams(NOUN,Name,accs,anim,femn,plur))]),
 MorphToken('Медведев',
            [31, 39),
            'RU',
            [Form('медведев', Grams(NOUN